# おまけ
というわけで foldRight と foldLeft を互いに書くことができるかを試すなかで、よくわからない部分を改にて、ここで実験してみます。

StackOverFlow にあった内容をちょとだけ。

In [13]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int =
        ints match {
            case Nil => 0
            case Cons(x, xs) => x + sum(xs)
        }
    
    def apply[A](as: A*): List[A] =
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    
    def tail[A](as: List[A]): List[A] =
        as match {
            case Nil => Nil
            case Cons(h, t) => t
        }
    
    def setHead[A](l: List[A], a: A): List[A] =
        l match {
            case Nil => apply(a)
            case Cons(_, t) => Cons(a, t)
        }
    
    def drop[A](l: List[A], n: Int): List[A] =
        if (n <= 0) l
        else l match {
            case Nil => Nil
            case Cons(_, t) => drop(t, n-1)
        }
    
    def dropWhile[A](l: List[A], f: A => Boolean): List[A] =
        l match {
            case Cons(h, t) => if (f(h)) dropWhile(t, f) else l
            case _ => l
        }
    
    def init[A](l: List[A]): List[A] =
        l match {
            case Nil => sys.error("empty")
            case Cons(_,Nil) => Nil
            case Cons(h,t) => Cons(h, init(t))
        }
 
    def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
          case Nil => z
          case Cons(x, xs) => f(x, foldRight(xs, z)(f))
    }
    
    def length[A](as: List[A]): Int =
        foldRight(as, 0)((x,y) => 1 + y)
    
    @annotation.tailrec
    def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B =
        as match {
            case Nil => z
            case Cons(x, xs) => foldLeft(xs, f(z, x))(f)
        }
    
    def reverse[A](as: List[A]): List[A] =
        foldLeft(as, Nil:List[A])((z, l) => Cons(l, z))
    
    def foldLeftViaFoldRight0[A,B](l: List[A], z: B)(f: (B,A) => B): B =
        foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)
    
    def foldLeftViaFoldRight1[A,B](l: List[A], z: B)(f: (B,A) => B): B =
        foldRight(l, identity[B] _)((a,g) => g compose (b => f(b,a)))(z)

    def foldLeftViaFoldRight2[A,B](l: List[A], z: B)
                                 (f: (B,A) => B): B =
    {
      def h(a: A, g: B => B): (B => B) =
          g compose ((x: B) => f(x,a))

      foldRight(l, identity[B] _)(h _)(z);
    }    
}


defined trait List
defined object Nil
defined class Cons
defined object List

ここで定義した foldLeftViaFoldRight は、もとの例を書き直している。
```Scala
    def foldLeftViaFoldRight[A,B](l: List[A], z: B)(f: (B,A) => B): B =
        foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)    
```

まず、(b:B) => b は identity[B] で、これを _ で受けてオブジェクト化している。
オブジェクト化しないと _ で受けろと怒られるので、それに従う。
```
Compilation Failed
Main.scala:127: missing arguments for method identity in object Predef;
follow this method with `_' if you want to treat it as a partially applied function
        foldRight(l, identity[B])((a,g) => g compose (b => f(b,a)))(z)
                             ^
```

結局、foldRight の定義の z:B' で、B' は B => B という型になっている。
foldRightが使う関数f は f(A, B') => B' つまり f(A, B=>B) => (B=>B) という型の関数をあたえなければいけないので、gの戻り値はBとすることができる。

で、g は b を引数としたとき先にもらっているaも利用した f(b,a) の結果を引数とする関数だ。
これが、以下の説明になるのか。

```Scala
def foldLeftViaFoldRight[A,B](l: List[A], z: B)
                             (f: (B,A) => B): B =
{
  def h(a: A, g: B => B): (B => B) =
    g compose ((x: B) => f(x,a));
  l.foldRight(identity[B] _)(h _)(z);
}
```
これを自前のList風に書き直すと。
```Scala
def foldLeftViaFoldRight2[A,B](l: List[A], z: B)
                             (f: (B,A) => B): B =
{
  def h(a: A, g: B => B): (B => B) =
      g compose ((x: B) => f(x,a))
  
  foldRight(l, identity[B] _)(h _)(z);
}
```
うまくコンパイルされている。
で、List(1,2) の時に、foldRight に沿って、書きなおす。

```
l は Cons(1, Cons(2,Nil)) なので h(1, foldRight(Cons(2,Nil), identity[B] _)(h _))

Cons(2,Nil) match Cons(2,Nil) なので h(1, h(2, foldRight(Nil, identitiy[B] _)(h _))

Nil match Nil なので h(1, h(2, identity[B] _))

つぎに、h をバラして、

h(1, identity[B] compose ((x:B) => f(x,2)))

identify[B] は y:B => y のことなので、

h(1, ((x:B) => f(x,2)))

((x:B) => f(x,2) compose ((x:B) => f(x,1)) )

後ろの x:B を前の x:B と区別がつくように y に直した後、
(y:B) => f(f(y,1), 2)

(y:B) という関数にz を引数で与えればよくて、

f(f(z, 1), 2)

f(a,b) = x + y
だとすると

(z + 1) + 2

となる。
```

ようやくわかったけど、複雑すぎる。これを一気にかくとオリジナルになる。
慣れ？